In [1]:
import pandas as pd
import re

## Display all the columns of the dataframe
pd.pandas.set_option('display.max_columns',None)

## load the file

In [2]:
df=pd.read_excel("adobe_dataset_train.xlsx")

In [3]:
df.head()

,id,date,likes,content,username,media,inferred company
0,1,2020-12-12 00:47:00,1,"Spend your weekend morning with a Ham, Egg, an...",TimHortonsPH,[Photo(previewUrl='https://pbs.twimg.com/media...,tim hortons
1,2,2018-06-30 10:04:20,2750,Watch rapper <mention> freestyle for over an H...,IndyMusic,[Photo(previewUrl='https://pbs.twimg.com/media...,independent
2,3,2020-09-29 19:47:28,57,Canadian Armenian community demands ban on mil...,CBCCanada,[Photo(previewUrl='https://pbs.twimg.com/media...,cbc
3,4,2020-10-01 11:40:09,152,"1st in Europe to be devastated by COVID-19, It...",MKWilliamsRome,[Photo(previewUrl='https://pbs.twimg.com/media...,williams
4,5,2018-10-19 14:30:46,41,Congratulations to Pauletha Butts of <mention>...,BGISD,[Photo(previewUrl='https://pbs.twimg.com/media...,independent


## From the above dataset some of the features like Id is not required

In [4]:
df.drop(columns=['id'], inplace=True)

## Remove all <"xyz"> from content

In [6]:
df['content'] = df['content'].apply(lambda x: re.sub(r'<.*?>', '', x))

## Remove dublicate rows

In [7]:
duplicate_rows = df[df.duplicated()]
if not duplicate_rows.empty:
    print("\nDuplicate rows:")
    print(duplicate_rows)
    df.drop_duplicates(inplace=True)
else:
    print("\nNo duplicate rows found.")


No duplicate rows found.


## Seperate hashtags from content

In [8]:
def extract_hashtags(content):
    hashtags = re.findall(r'#(\w+)', content)
    if hashtags:
        cleaned_hashtags = [hashtag.replace('#', '') if '#' in hashtag else hashtag for hashtag in hashtags]
        return ' '.join(cleaned_hashtags)
    else:
        return 'no_tags'

In [9]:
# Add a new column for cleaned hashtags
df['cleaned_hashtags'] =df['content'].apply(extract_hashtags)

# Remove hashtags from content column
df['content'] =df['content'].apply(lambda x: re.sub(r'(?<=\s)#', '', x))
# Display the updated dataframe


## Add a column which speciphy Media type

In [10]:
# Extract media types from the 'media' column
df['media_type'] = df['media'].str.extract(r"\[([A-Za-z]+)\(")

In [11]:
df.head()

,date,likes,content,username,media,inferred company,cleaned_hashtags,media_type
0,2020-12-12 00:47:00,1,"Spend your weekend morning with a Ham, Egg, an...",TimHortonsPH,[Photo(previewUrl='https://pbs.twimg.com/media...,tim hortons,TimHortonsPH,Photo
1,2018-06-30 10:04:20,2750,Watch rapper freestyle for over an HOUR,IndyMusic,[Photo(previewUrl='https://pbs.twimg.com/media...,independent,no_tags,Photo
2,2020-09-29 19:47:28,57,Canadian Armenian community demands ban on mil...,CBCCanada,[Photo(previewUrl='https://pbs.twimg.com/media...,cbc,no_tags,Photo
3,2020-10-01 11:40:09,152,"1st in Europe to be devastated by COVID-19, It...",MKWilliamsRome,[Photo(previewUrl='https://pbs.twimg.com/media...,williams,no_tags,Photo
4,2018-10-19 14:30:46,41,Congratulations to Pauletha Butts of ! She was...,BGISD,[Photo(previewUrl='https://pbs.twimg.com/media...,independent,excellenceisworththeeffort,Photo


## Find missing values

In [12]:
missing_values = df.isnull().sum()
print("Missing values:")
print(missing_values)

Missing values:
date                0
likes               0
content             0
username            0
media               0
inferred company    0
cleaned_hashtags    0
media_type          0
dtype: int64


## NOTE: Emojies are not removed because they can provide a significant role in sentiment analysis

## save the cleaned csv

In [13]:
df.to_excel('adobe_dataset_cleaned_train.xlsx', index=False)